<a href="https://colab.research.google.com/github/MengOonLee/WebScrapy/blob/master/Groceries/Lotus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install --no-cache-dir -qU scrapy selenium

In [17]:
from selenium import webdriver
import scrapy
import time

options = webdriver.chrome.options.Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--enable-javascript")
driver = webdriver.Chrome(options=options)

url = "https://www.lotuss.com.my/en/category/grocery"
driver.get(url)
time.sleep(10)
selector = scrapy.Selector(text=driver.page_source.encode("utf-8"))
products = selector.css("div.product-grid-item a#product-title::text")\
    .getall()
driver.close()

products

['GARDENS CHILLI SAUCE 710G',
 'MAMEE PREMIUM MI TARIK VEGETARIAN 5X80G',
 'JATI 100 BERAS WANGI AAA 10KG',
 'MAMEE PREMIUM MI TARIK CHICKEN 5X79G',
 'MAMEE PREMIUM MI TARIK CURRY 5X80G',
 'MAMEE PREMIUM MI TARIK VEGE CURRY 5X78G',
 'MAMEE PREMIUM MI TARIK DUCK 5X81G',
 'JACOBS CREAM CRACKER MP 504G',
 'JACOBS WEETAMEAL MULTIPACK 502G',
 'KELLOGGS COCO POPS 350G',
 'NESTLE HONEY STARS ECONOPACK 450G',
 'RASAKU COCONUT MILK 200MLX6',
 'SKIPPY P.B. CHOCOLATE STRIPES 530G',
 'CADBURY HAZEL NUTS 160G',
 'NAXNAX CHOCOLATE MARIE BISCUITS 300G']

In [ ]:
%%writefile Lotus.py
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import scrapy
from scrapy.crawler import CrawlerProcess

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

class LotusSpider(scrapy.Spider):
    name = 'Lotus'

    def start_requests(self):
        urls = [
            'https://quotes.toscrape.com'
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse,
                meta={'url': url})

    def parse(self, response):
        url = response.meta['url']

        driver.get(url)

        element = driver.find_element(By.CSS_SELECTOR, 'div.quote')
        print(element.text)

        driver.close()

process = CrawlerProcess()
process.crawl(LotusSpider)
process.start()

Overwriting Lotus.py


In [ ]:
%%bash
python Lotus.py